In [ ]:
import pandas as pd
import json
import numpy as np

: 

# 整合数据集

In [19]:
# datasets = ["assist2009", "algebra2005", "bridge2algebra2006", "nips_task34", "ednet", "peiyou", "ednet5w"]
# datasets_dic = {"assist2009": 0, "algebra2005": 1, "bridge2algebra2006": 2, "nips_task34": 3, "ednet": 4, "peiyou": 5, "ednet5w": 6}
datasets= [ "ednet_all"]
datasets_dic = {"ednet_all": 0}

In [20]:
new_data = {"fold":[], "uid":[], "questions":[], "concepts":[], "responses":[], "dataset":[], "timestamps":[]}

In [21]:
for dataset in datasets:
    df_train = pd.read_csv(f"../data/{dataset}/train_valid_quelevel.csv")
    df_test = pd.read_csv(f"../data/{dataset}/test_quelevel.csv")
    df = pd.concat([df_train, df_test])
    
    data_info_ = dict()
    with open(f"../data/{dataset}/keyid2idx.json", "r") as f:
        data_info = json.load(f)
        for key in data_info:
            data_info_.setdefault(key,dict())
            try:
                for item in data_info[key]:
                    data_info_[key][data_info[key][item]] = item
            except:
                print(f"{key}")
                continue
    for i,row in df.iterrows():
        uid = data_info_["uid"][row["uid"]]
        uid = uid.replace("(","")
        uid = uid.replace("\uFF08", "")
        # print(uid)
        questions = row["questions"].split(",")
        # print(f"questions:{questions}")
        concepts = row["concepts"].split(",")
        # print(f"concepts:{concepts}")
        questions = [data_info_["questions"][int(q)] for q in questions]
        new_concepts = []
        for ccc in concepts:
            ccc = ccc.split("_")
            concept = [data_info_["concepts"][int(c)] for c in ccc]
            concept = "_".join(concept)
            new_concepts.append(concept)
        new_data["fold"].append(row["fold"])
        new_data["uid"].append(uid)
        new_data["questions"].append(",".join(questions))
        new_data["concepts"].append(",".join(new_concepts))
        new_data["responses"].append(row["responses"])
        new_data["dataset"].append(dataset)
        if "timestamps" in row:
            new_data["timestamps"].append(row["timestamps"])
        else:
            new_data["timestamps"].append(",".join([str(i) for i in range(len(questions))]))

max_concepts


In [23]:
new_df = pd.DataFrame(new_data)

In [ ]:
new_df.to_csv(f"../data/ednet_all/train_valid_quelevel_pretrain.csv", index=None)

In [ ]:
new_df[:5]

In [ ]:
df_sort = pd.read_csv('../data/ednet_all/train_valid_quelevel_pretrain.csv')
df_sort = df_sort.sort_values(by='uid',ascending = True)

In [ ]:
df_sort[:5]

# 完成数据映射

In [ ]:
def id_mapping_que(df):
    id_keys = ["questions", "concepts", "uid"]
    dres = dict()
    dkeyid2idx = dict()
    # print(f"df.columns: {df.columns}")
    flag = True #判定数据集有没有变化
    pre_data = "ednet"
    for key in df.columns:
        if key not in id_keys:
            dres[key] = df[key]
    for i, row in df.iterrows():
        dataset = row["dataset"]
        if dataset not in ["ednet5w", "ednet_all"]:
            for j,key in enumerate(id_keys):
                if key not in df.columns:
                    continue
                if key == "timestamps":
                    dres[key].append(row[key])
                else:
                    dkeyid2idx.setdefault(key, dict())
                    dkeyid2idx[key].setdefault(dataset, dict())
                    dres.setdefault(key, [])
                    curids = []
                    # print(f"dkeyid2idx:{dkeyid2idx}")
                    # {'questions': {'ednet': {}}}
                    for id in row[key].split(","):
                        sub_ids = id.split('_')
                        sub_curids = []
                        for sub_id in sub_ids:
                            if sub_id not in dkeyid2idx[key][dataset]:
                                if list(dkeyid2idx[key][dataset].values()) == []: #到新的数据集
                                    if dataset == "ednet":
                                        dkeyid2idx[key][dataset][sub_id] = 0
                                        flag = False
                                    else:
                                        # print(f"key:{key}, dataset:{dataset}, pre_data:{pre_data}")
                                        # print(f"error:{dkeyid2idx[key][pre_data]}")
                                        dkeyid2idx[key][dataset][sub_id] = list(dkeyid2idx[key][pre_data].values())[-1]+1
                                else: #原本数据集
                                    dkeyid2idx[key][dataset][sub_id] = list(dkeyid2idx[key][dataset].values())[-1]+1
                                # dkeyid2idx[key][dataset][sub_id] = cnt+1
                            sub_curids.append(str(dkeyid2idx[key][dataset][sub_id]))
                            # cnt += 1
                        curids.append("_".join(sub_curids))
                    dres[key].append(",".join(curids))
            pre_data = dataset
        elif dataset == "ednet5w":
            for j,key in enumerate(id_keys):
                if key not in df.columns:
                    continue
                if key == "timestamps":
                    dres[key].append(row[key])
                else:
                    dkeyid2idx.setdefault(key, dict())
                    dkeyid2idx[key].setdefault(dataset, dict())
                    dres.setdefault(key, [])
                    curids = []
                    # print(f"dkeyid2idx:{dkeyid2idx}")
                    # {'questions': {'ednet': {}}}
                    for id in row[key].split(","):
                        sub_ids = id.split('_')
                        sub_curids = []
                        for sub_id in sub_ids:
                            if sub_id in dkeyid2idx[key]["ednet"]:
                                sub_curids.append(str(dkeyid2idx[key]["ednet"][sub_id]))
                            elif sub_id not in dkeyid2idx[key]["ednet"] and sub_id not in dkeyid2idx[key][dataset]:
                                if list(dkeyid2idx[key][dataset].values()) == []: #到新的数据集
                                    dkeyid2idx[key][dataset][sub_id] = list(dkeyid2idx[key]["ednet"].values())[-1]+1
                                else: #原本数据集
                                    dkeyid2idx[key][dataset][sub_id] = list(dkeyid2idx[key][dataset].values())[-1]+1
                                sub_curids.append(str(dkeyid2idx[key][dataset][sub_id]))
                            elif sub_id in dkeyid2idx[key][dataset]:
                                sub_curids.append(str(dkeyid2idx[key][dataset][sub_id]))
                            # cnt += 1
                        curids.append("_".join(sub_curids))
                    dres[key].append(",".join(curids))
        else:
            for j,key in enumerate(id_keys):
                if key not in df.columns:
                    continue
                if key == "timestamps":
                    dres[key].append(row[key])
                else:
                    dkeyid2idx.setdefault(key, dict())
                    dkeyid2idx[key].setdefault(dataset, dict())
                    dres.setdefault(key, [])
                    curids = []
                    # print(f"dkeyid2idx:{dkeyid2idx}")
                    for id in row[key].split(","):
                        sub_ids = id.split('_')
                        sub_curids = []
                        for sub_id in sub_ids:
                            if sub_id in dkeyid2idx[key]["ednet"]:
                                sub_curids.append(str(dkeyid2idx[key]["ednet"][sub_id]))
                            elif sub_id not in dkeyid2idx[key]["ednet"] and sub_id not in dkeyid2idx[key]["ednet5w"] and sub_id not in dkeyid2idx[key][dataset]:
                                if list(dkeyid2idx[key][dataset].values()) == []: #到新的数据集
                                    dkeyid2idx[key][dataset][sub_id] = list(dkeyid2idx[key]["ednet5w"].values())[-1]+1
                                else: #原本数据集
                                    dkeyid2idx[key][dataset][sub_id] = list(dkeyid2idx[key][dataset].values())[-1]+1
                                sub_curids.append(str(dkeyid2idx[key][dataset][sub_id]))
                            elif sub_id in dkeyid2idx[key][dataset]:
                                sub_curids.append(str(dkeyid2idx[key][dataset][sub_id]))
                            # cnt += 1
                        curids.append("_".join(sub_curids))
                    dres[key].append(",".join(curids))
    finaldf = pd.DataFrame(dres)
    return finaldf, dkeyid2idx

# ednet_all数据集

In [24]:
def id_mapping_que(df):
    id_keys = ["questions", "concepts", "uid"]
    dres = dict()
    dkeyid2idx = dict()
    # print(f"df.columns: {df.columns}")
    flag = True #判定数据集有没有变化
    pre_data = "ednet_all"
    for key in df.columns:
        if key not in id_keys:
            dres[key] = df[key]
    for i, row in df.iterrows():
        dataset = row["dataset"]
        if dataset == "ednet_all":
            for j,key in enumerate(id_keys):
                if key not in df.columns:
                    continue
                if key == "timestamps":
                    dres[key].append(row[key])
                else:
                    dkeyid2idx.setdefault(key, dict())
                    dkeyid2idx[key].setdefault(dataset, dict())
                    dres.setdefault(key, [])
                    curids = []
                    # print(f"dkeyid2idx:{dkeyid2idx}")
                    # {'questions': {'ednet': {}}}
                    for id in row[key].split(","):
                        sub_ids = id.split('_')
                        sub_curids = []
                        for sub_id in sub_ids:
                            if sub_id not in dkeyid2idx[key][dataset]:
                                if list(dkeyid2idx[key][dataset].values()) == []: #到新的数据集
                                    if dataset == "ednet_all":
                                        dkeyid2idx[key][dataset][sub_id] = 0
                                        flag = False
                                    else:
                                        # print(f"key:{key}, dataset:{dataset}, pre_data:{pre_data}")
                                        # print(f"error:{dkeyid2idx[key][pre_data]}")
                                        dkeyid2idx[key][dataset][sub_id] = list(dkeyid2idx[key][pre_data].values())[-1]+1
                                else: #原本数据集
                                    dkeyid2idx[key][dataset][sub_id] = list(dkeyid2idx[key][dataset].values())[-1]+1
                                # dkeyid2idx[key][dataset][sub_id] = cnt+1
                            sub_curids.append(str(dkeyid2idx[key][dataset][sub_id]))
                            # cnt += 1
                        curids.append("_".join(sub_curids))
                    dres[key].append(",".join(curids))
            pre_data = dataset
    finaldf = pd.DataFrame(dres)
    return finaldf, dkeyid2idx

In [25]:
finaldf, dkeyid2idx = id_mapping_que (new_df)

KeyboardInterrupt: 

In [ ]:
with open(f"../data/pretrain/keyid2idx.json", "w") as f:
    json.dump(dkeyid2idx, f)

In [ ]:
with open(f"../data/ednet_all/keyid2idx.json", "w") as f:
    json.dump(dkeyid2idx, f)

In [ ]:
list(dkeyid2idx['questions']['ednet'].values())[-1]

In [ ]:
# len(list(dkeyid2idx['questions']['ednet5w'].values()))
len(list(dkeyid2idx['questions']['ednet_all'].values()))

In [ ]:
for i, row in enumerate(finaldf["questions"]):
    row_ = row.split(",")
    try:
        
        row_ = [int(x) for x in row_]
    except:
        print(i,row)
        break

In [ ]:
len(dkeyid2idx['questions']['bridge2algebra2006'])

In [ ]:
len(dkeyid2idx['questions']['algebra2005'])

In [ ]:
list(dkeyid2idx['questions']['ednet_all'].values())[-1]

In [ ]:
finaldf.head(5)

# 计算一题对应最多知识点数量

In [ ]:
def get_max_concepts(df):
    max_concepts = 1
    for i, row in df.iterrows():
        cs = row["concepts"].split(",")
        num_concepts = max([len(c.split("_")) for c in cs])
        if num_concepts >= max_concepts:
            max_concepts = num_concepts
    return max_concepts

In [ ]:
max_concepts = get_max_concepts(new_df)

In [ ]:
max_concepts

# 数据统计

In [ ]:
def calStatistics(df, stares, key):
    allin, allselect = 0, 0
    allqs, allcs = set(), set()
    for i, row in df.iterrows():
        rs = row["responses"].split(",")
        curlen = len(rs) - rs.count("-1")
        allin += curlen
        if "selectmasks" in row:
            ss = row["selectmasks"].split(",")
            slen = ss.count("1")
            allselect += slen
        if "concepts" in row:
            cs = row["concepts"].split(",")
            fc = list()
            for c in cs:
                cc = c.split("_")
                fc.extend(cc)
            curcs = set(fc) - {"-1"}
            allcs |= curcs
        if "questions" in row:
            qs = row["questions"].split(",")
            curqs = set(qs) - {"-1"}
            allqs |= curqs
    stares.append(",".join([str(s) for s in [key, allin, df.shape[0], allselect]]))
    return allin, allselect, len(allqs), len(allcs), df.shape[0]

In [ ]:
finaldf[finaldf.dataset=="ednet_all"]

In [ ]:
ins, ss, qs, cs, seqnum = calStatistics(df=finaldf, stares=[], key="original train+valid question level")

In [ ]:
ins, ss, qs, cs, seqnum

In [ ]:
for data in datasets:
    final_sub_df = finaldf[finaldf.dataset==data]
    ins_, ss_, qs_, cs_, seqnum_ = calStatistics(df=final_sub_df, stares=[], key="original train+valid question level")
    print(f"dataset:{data}, ins_:{ins_}, ss_:{ss_}, qs_:{qs_}, cs_:{cs_}, seqnum_:{seqnum_}")

# 把数据集划分成sequence

In [ ]:

def generate_sequences(df, effective_keys, min_seq_len=3, maxlen = 200, pad_val = -1):
    save_keys = list(effective_keys) + ["selectmasks"]
    dres = {"selectmasks": []}

    dropnum = 0
    for i, row in df.iterrows():
        dcur = save_dcur(row, effective_keys)

        rest, lenrs = len(dcur["responses"]), len(dcur["responses"])
        j = 0
        while lenrs >= j + maxlen:   
            rest = rest - (maxlen)
            for key in effective_keys:
                dres.setdefault(key, [])
                if key not in ONE_KEYS:
                    dres[key].append(",".join(dcur[key][j: j + maxlen]))#[str(k) for k in dcur[key][j: j + maxlen]]))
                else:
                    dres[key].append(dcur[key])
            dres["selectmasks"].append(",".join(["1"] * maxlen))

            j += maxlen
        if rest < min_seq_len:# delete sequence len less than min_seq_len
            dropnum += rest
            continue
        
        pad_dim = maxlen - rest
        for key in effective_keys:
            dres.setdefault(key, [])
            if key not in ONE_KEYS:
                paded_info = np.concatenate([dcur[key][j:], np.array([pad_val] * pad_dim)])
                dres[key].append(",".join([str(k) for k in paded_info]))
            else:
                if key == "dataset":
                    dres[key].append(datasets_dic[dcur[key]])
                else:
                    dres[key].append(dcur[key])
        dres["selectmasks"].append(",".join(["1"] * rest + [str(pad_val)] * pad_dim))
    
    # after preprocess data, report
    dfinal = dict()
    for key in ALL_KEYS:
        if key in save_keys:
            dfinal[key] = dres[key]
    finaldf = pd.DataFrame(dfinal)
    print(f"dropnum: {dropnum}")
    return finaldf

In [ ]:
def save_dcur(row, effective_keys):
    dcur = dict()
    for key in effective_keys:
        if key not in ONE_KEYS:
            dcur[key] = row[key].split(",")#[int(i) for i in row[key].split(",")]
        else:
            dcur[key] = row[key]
    return dcur

In [ ]:
ONE_KEYS = ["fold", "uid","dataset"]

In [ ]:
ALL_KEYS = ["fold", "uid", "dataset", "questions", "concepts", "responses", "timestamps", "usetimes", "selectmasks", "is_repeat", "qidxs", "rest", "orirow","cidxs"]

In [ ]:
split_seqs = generate_sequences(df=finaldf, effective_keys={"uid", "dataset", "questions","concepts","responses","fold", "timestamps"}, min_seq_len=3, maxlen=200)

In [ ]:
ins, ss, qs, cs, seqnum = calStatistics(df=split_seqs, stares=[], key="train+valid sequences question level")

In [ ]:
ins, ss, qs, cs, seqnum

In [ ]:
ins, ss, qs, cs, seqnum

In [ ]:
split_seqs.shape

In [ ]:
finaldf.shape

In [ ]:
split_seqs

In [ ]:
test_seqs = split_seqs
test_seqs

In [ ]:
test_seqs['dataset'] = test_seqs['dataset'].apply(lambda x: datasets_dic[x] if x in datasets_dic else x)

In [ ]:
test_seqs

In [ ]:
test_seqs.to_csv(f"../data/ednet_all/train_valid_sequences_quelevel_200.csv", index=None)

In [ ]:
finaldf.to_csv(f"../data/ednet_all/train_valid_quelevel.csv", index=None)

# 完成测试集映射

In [ ]:
datasets = ["assist2009", "algebra2005", "bridge2algebra2006", "nips_task34", "ednet", "ednet5w", "peiyou"]

In [ ]:
def map_dataset(datasets):
    new_data = {"fold":[], "uid":[], "questions":[], "concepts":[], "responses":[], "dataset":[], "timestamps":[]}
    for dataset in datasets:
        datapath = f"../data/{dataset}/test_quelevel.csv"
        df = pd.read_csv(datapath)
        data_info_ = dict()
        with open(f"../data/{dataset}/keyid2idx.json") as f:
            data_info = json.load(f)
            for key in data_info:
                data_info_.setdefault(key,dict())
                try:
                    for item in data_info[key]:
                        data_info_[key][data_info[key][item]] = item
                except:
                    print(f"{key}")
                    continue
        for i,row in df.iterrows():
            uid = data_info_["uid"][row["uid"]]
            uid = uid.replace("(","")
            questions = row["questions"].split(",")
            # print(f"questions:{questions}")
            concepts = row["concepts"].split(",")
            # print(f"concepts:{concepts}")
            questions = [data_info_["questions"][int(q)] for q in questions]
            new_concepts = []
            for ccc in concepts:
                ccc = ccc.split("_")
                concept = [data_info_["concepts"][int(c)] for c in ccc]
                concept = "_".join(concept)
                new_concepts.append(concept)
            new_data["fold"].append(row["fold"])
            new_data["uid"].append(uid)
            new_data["questions"].append(",".join(questions))
            new_data["concepts"].append(",".join(new_concepts))
            new_data["responses"].append(row["responses"])
            new_data["dataset"].append(dataset)
            if "timestamps" in row:
                new_data["timestamps"].append(row["timestamps"])
            else:
                new_data["timestamps"].append(",".join([str(i) for i in range(len(questions))]))
    return new_data

In [ ]:
new_data = map_dataset(datasets)

In [ ]:
new_df = pd.DataFrame(new_data)

In [ ]:
new_df[-5:]

In [ ]:
with open(f"../data/pretrain/keyid2idx.json","r") as f:
    dkeyid2idx = json.load(f)

## 除ednet5w

In [ ]:
# datasets = ["assist2009", "algebra2005", "bridge2algebra2006", "nips_task34", "ednet", "peiyou"]
datasets = ["ednet_all"]
for dataset in datasets:
    if dataset not in ["ednet5w"]:
        sub_df = new_df[new_df.dataset==dataset]
        new_map_data = {"fold":[], "uid":[], "dataset":[], "questions":[], "concepts":[], "responses":[], "timestamps":[]}
        for i,row in sub_df.iterrows():
            questions = [str(dkeyid2idx["questions"][dataset][x]) for x in row["questions"].split(",")]
            concepts = row["concepts"].split(",")
            new_concepts = []
            for ccc in concepts:
                ccc = ccc.split("_")
                concept = [str(dkeyid2idx["concepts"][dataset][c]) for c in ccc]
                # concept = [str(dkeyid2idx.get("concepts").get(dataset).get(c,len(dkeyid2idx['concepts'][dataset]))) for c in ccc]
                concept = "_".join(concept)
                new_concepts.append(concept)
            new_map_data["fold"].append(row["fold"])
            new_map_data["uid"].append(row["uid"])
            new_map_data["dataset"].append(dataset)
            new_map_data["questions"].append(",".join(questions))
            new_map_data["concepts"].append(",".join(new_concepts))
            new_map_data["responses"].append(row["responses"])
            new_map_data["timestamps"].append(row["timestamps"])
            new_map_df = pd.DataFrame(new_map_data)
            new_map_df.to_csv(f"../data/{dataset}/test_quelevel_pretrain.csv")

In [ ]:
new_map_df[:5]

In [ ]:
# datasets = ["assist2009", "algebra2005", "bridge2algebra2006", "nips_task34", "ednet", "peiyou"]
# for dataset in datasets:
#     sub_df = new_df[new_df.dataset==dataset]
#     new_map_data = {"fold":[], "uid":[], "questions":[], "concepts":[], "responses":[], "timestamps":[]}
#     for i,row in sub_df.iterrows():
#         questions = [str(dkeyid2idx["questions"][dataset][x]) for x in row["questions"].split(",")]
#         concepts = row["concepts"].split(",")
#         dataset = datasets_dic[dataset]
#         new_concepts = []
#         for ccc in concepts:
#             ccc = ccc.split("_")
#             concept = [str(dkeyid2idx["concepts"][dataset][c]) for c in ccc]
#             # concept = [str(dkeyid2idx.get("concepts").get(dataset).get(c,len(dkeyid2idx['concepts'][dataset]))) for c in ccc]
#             concept = "_".join(concept)
#             new_concepts.append(concept)
#         new_map_data["fold"].append(row["fold"])
#         new_map_data["uid"].append(row["uid"])
#         new_map_data["questions"].append(",".join(questions))
#         new_map_data["concepts"].append(",".join(new_concepts))
#         new_map_data["responses"].append(row["responses"])
#         new_map_data["timestamps"].append(row["timestamps"])
#         new_map_df = pd.DataFrame(new_map_data)
#         new_map_df.to_csv(f"../data/{dataset}/test_quelevel_pretrain.csv")

In [ ]:
datasets = ["ednet5w"]
for dataset in datasets:
    sub_df = new_df[new_df.dataset==dataset]
    new_map_data = {"fold":[], "uid":[], "dataset":[], "questions":[], "concepts":[], "responses":[], "timestamps":[]}
    for i,row in sub_df.iterrows():
        questions = []
        for x in row["questions"].split(","):
            if x in dkeyid2idx["questions"]["ednet"]:
                questions.append(str(dkeyid2idx["questions"]["ednet"][x]))
            else:
                questions.append(str(dkeyid2idx["questions"][dataset][x]))
        
        concept = []
        concepts = row["concepts"].split(",")
        new_concepts = []
        for ccc in concepts:
            ccc = ccc.split("_")
            concept = []
            for c in ccc:
                if c in dkeyid2idx['concepts']["ednet"]:
                    concept.append(str(dkeyid2idx["concepts"]["ednet"][c]))
                else:
                    concept.append(str(dkeyid2idx["concepts"][dataset][c]))
            concept = "_".join(concept)
            new_concepts.append(concept)
        new_map_data["fold"].append(row["fold"])
        new_map_data["uid"].append(row["uid"])
        new_map_data["dataset"].append(dataset)
        new_map_data["questions"].append(",".join(questions))
        new_map_data["concepts"].append(",".join(new_concepts))
        new_map_data["responses"].append(row["responses"])
        new_map_data["timestamps"].append(row["timestamps"])
        new_map_df = pd.DataFrame(new_map_data)
        new_map_df.to_csv(f"../data/{dataset}/test_quelevel_pretrain.csv")

In [ ]:
new_map_df

## 生成window测试集

In [ ]:
def generate_window_sequences(df, effective_keys, maxlen=200, pad_val=-1):
    save_keys = list(effective_keys) + ["selectmasks"]
    dres = {"selectmasks": []}
    for i, row in df.iterrows():
        dcur = save_dcur(row, effective_keys)
        lenrs = len(dcur["responses"])
        if lenrs > maxlen:
            for key in effective_keys:
                dres.setdefault(key, [])
                if key not in ONE_KEYS:
                    dres[key].append(",".join(dcur[key][0: maxlen]))#[str(k) for k in dcur[key][0: maxlen]]))
                else:
                    dres[key].append(dcur[key])
            dres["selectmasks"].append(",".join(["1"] * maxlen))
            for j in range(maxlen+1, lenrs+1):
                for key in effective_keys:
                    dres.setdefault(key, [])
                    if key not in ONE_KEYS:
                        dres[key].append(",".join([str(k) for k in dcur[key][j-maxlen: j]]))
                    else:
                        if key == "dataset":
                            dres[key].append(datasets_dic[dcur[key]])
                        else:
                            dres[key].append(dcur[key])
                dres["selectmasks"].append(",".join([str(pad_val)] * (maxlen - 1) + ["1"]))
        else:
            for key in effective_keys:
                dres.setdefault(key, [])
                if key not in ONE_KEYS:
                    pad_dim = maxlen - lenrs
                    paded_info = np.concatenate([dcur[key][0:], np.array([pad_val] * pad_dim)])
                    dres[key].append(",".join([str(k) for k in paded_info]))
                else:
                    if key == "dataset":
                        dres[key].append(datasets_dic[dcur[key]])
                    else:
                        dres[key].append(dcur[key])
            dres["selectmasks"].append(",".join(["1"] * lenrs + [str(pad_val)] * pad_dim))
    
    dfinal = dict()
    for key in ALL_KEYS:
        if key in save_keys:
            # print(f"key: {key}, len: {len(dres[key])}")
            dfinal[key] = dres[key]
    finaldf = pd.DataFrame(dfinal)
    return finaldf

In [ ]:
datasets = ["assist2009", "algebra2005", "bridge2algebra2006", "nips_task34", "ednet", "peiyou", "ednet5w"]
for data in datasets:
    test_df = pd.read_csv(f"../data/{data}/test_quelevel_pretrain.csv")
    test_window_seqs = generate_window_sequences(test_df, list({"uid","dataset","questions","concepts","responses","fold"}), maxlen=200)
    test_window_seqs['dataset'] = test_window_seqs['dataset'].apply(lambda x: datasets_dic[x] if x in datasets_dic else x)
    test_window_seqs.to_csv(f"../data/{data}/test_window_sequences_quelevel_pretrain_200.csv")

In [ ]:
test_df = pd.read_csv(f"../data/ednet_all/test_quelevel_pretrain.csv")
test_window_seqs = generate_window_sequences(test_df, list({"uid","dataset","questions","concepts","responses","fold"}), maxlen=200)
test_window_seqs['dataset'] = test_window_seqs['dataset'].apply(lambda x: datasets_dic[x] if x in datasets_dic else x)
test_window_seqs.to_csv(f"../data/ednet_all/test_window_sequences_quelevel_200.csv")

In [ ]:
test_window_seqs

In [ ]:
test_df[:5]

In [ ]:
test_window_seqs = generate_window_sequences(test_df, list({"uid","questions","concepts","responses","fold"}), maxlen=1024)

In [ ]:
test_window_seqs

In [ ]:
import matplotlib.pyplot as plt

# 你的数据
data = [229471, 513430, 1463212, 1109911, 462626, 4719979, 4446825]

# 数据标签
labels = ['229471', '513430', '1463212', '1109911', '462626', '4719979', '4446825']

# 创建饼图
plt.pie(data, labels = labels, autopct='%1.1f%%')

# 添加标题
plt.title('数据饼图')

# 显示图表
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# 你的数据
data = [229471, 513430, 1463212, 1109911, 462626, 4719979, 4446825]

# 数据标签
labels = ["AS2009", "AL2005", "BD2006", "NIPS34", "EdNet", "EdNet5w", "Peiyou"]

# 创建饼图
plt.pie(data, labels = labels, autopct='%1.1f%%')

# 添加标题
plt.title('Pretrain data distribution')

plt.savefig("pie_chart.pdf", format='pdf')

# 显示图表
plt.show()


In [ ]:
path = "/root/autodl-nas/project/pykt_nips2022/examples/best_model_path/ednet5w/"

In [ ]:
import os

In [ ]:
models = os.listdir(path)

In [ ]:
for model in models:
    model_path = os.listdir(os.path.join(path, model))
    for best_path in model_path:
        print("- "+os.path.join(path, model, best_path))

In [ ]:
# 读取csv文件
train=pd.read_csv("../data/ednet_all/train_valid_sequences_quelevel_200.csv")

# 检查"timestamps"列的每一个元素的长度是否都为200
is_all_200 = train['timestamps'].apply(lambda x: len(x.split(",")) == 200).all()

print(is_all_200)
